In [1]:
import os
import pdf2image
from pdf2image import convert_from_path
import cv2
import matplotlib.pyplot as plt
import PyPDF2
import PIL
from PIL import Image, ImageTk
import numpy as np
import pytesseract
from pytesseract import Output
pytesseract.pytesseract.tesseract_cmd="/home/aman/anaconda3/envs/tallyInvoiceParser.env/bin/tesseract"
os.environ['TESSDATA_PREFIX'] = "/home/aman/anaconda3/envs/tallyInvoiceParser.env/share/tessdata"
os.environ['MLIR_CRASH_REPRODUCER_DIRECTORY']='tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269'
import import_ipynb
from textPreProcessor import *
from imagePreProcessor import *
from CreateDatasetForDateModel import *
from datetime import datetime
import math
from keras.models import load_model

importing Jupyter notebook from textPreProcessor.ipynb


2023-06-08 16:04:12.061905: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Original Date ----> After Processing 

5/12/22      ---->      5-12-2022
2/2/2023      ---->      2-2-2023
5:12-2023      ---->      5-12-2023
5,May,2023      ---->      5-05-2023
5 May,2023      ---->      5-05-2023
October 24,2022      ---->      10-24-2022
importing Jupyter notebook from imagePreProcessor.ipynb
importing Jupyter notebook from CreateDatasetForDateModel.ipynb


In [2]:
def preProcessImage(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    blur = cv2.GaussianBlur(gray, (3,3),1)
    threshold_img = cv2.adaptiveThreshold(blur,255,1,1,11,2)
    return threshold_img

In [8]:
def processForInput(df):
    colsToDrop = ["left", "top", "width", "height", "text", "index"]
    d = df.drop(columns=colsToDrop)
    d["conf"]=d["conf"]/100
    d["x"]=d["x"]/1000
    d["y"]=d["y"]/1000
    return d.to_numpy()

In [56]:
def predict(filePath):
    f = open(filePath, 'rb')
    readpdf = PyPDF2.PdfReader(f)
    totalpages = len(readpdf.pages)
    if totalpages==1:
        image = convert_from_path(filePath)
    else:
        return
    image = np.array(image[0])
    image = preProcessImage(image)
    print(type(image))
    data = pytesseract.image_to_data(image, output_type=Output.DATAFRAME)
    df = pd.DataFrame(data)
    preProcessDataFrame(df)
    dateDF, dates = extractDateDataFrame(df)
    addNeighbours(df)
    indexes = findDateDF(dates, dateDF)
    resultDF = df.copy()
    dropIndexes(indexes, resultDF)
    resultDF.reset_index(inplace=True)
    texts = []
    inputs = []
    for index in resultDF.index:
        texts.append(resultDF["text"][index])
        inputs.append(processForInput(resultDF))
    predictions = {}
    model = load_model("/home/aman/Documents/Tally/DocumentAI/Code/Tesseract-Model/Models/tesseractv1.h5")
    for i in range(len(inputs)):
        text = texts[i]
        input = inputs[i]
        pred = model.predict(input)
        predictions[text]=max(pred)
    if len(predictions)==0:
        return None
    res = max(zip(predictions.values(), predictions.keys()))[1]
    return res
    

In [57]:
os.chdir("/home/aman/Documents/Tally/DocumentAI/Data/SampleInvoices2/")

In [58]:
dir = os.listdir("/home/aman/Documents/Tally/DocumentAI/Data/SampleInvoices2/")

In [73]:
pdfPath = "/home/aman/Documents/Tally/DocumentAI/Data/SampleInvoices2/Bisleri-internation-bill-Dec-22.pdf"

In [74]:
print(predict(pdfPath))

<class 'numpy.ndarray'>


<string>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<string>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


1/1 [==============================] - 0s 17ms/step
31-12-22
